# General tip 3: Use vector art where appropriate

A **raster image** is a 2d matrix of colour values. Each matrix entry is called a pixel.

A **vector image** is a set of instructions for drawing.

Rendering images on screen or in print often requires a rasterisation step.
The main advantage of vector art is that it **delays rasterisation** until the very last step.
This avoids lossy intermediate resampling, and allows rasterisation to be carried out by a specialised device (e.g. a display driver or a printer) that has knowledge of the final pixel size.

For example, the same vector art might be rendered quite differently on a small phone screen than on a large screen.
Printers in particular, are full of little hacks and tweaks that allow them to obtain much nicer results.
In some cases, rasterisation can even be avoided altogether, e.g. with old-school [plotters](https://en.wikipedia.org/wiki/Plotter), or [PostScript fonts](https://en.wikipedia.org/wiki/PostScript_fonts).


### 3.1 The appropriate format depends on the data

If you start from raster data, e.g. a photograph, then stick with raster formats.

If you start from vector data, e.g. a set of points or a collection of lines, then use vector formats.

In some cases it makes sense to combine vector and raster art (which many vector formats allow).
For example, you can add vector annotations to a photograph.
Conversly, if you have vector data but find that the vector instructions take very long to execute (for example because you are showing a huge amount of data) it can be helpful to rasterise part of the image early.

### 3.2 Humans are not scale-invariant

Resizing a raster image requires resampling, which usually means a loss of quality.
Vector art can be scaled up and down without loss.

However, line widths and font sizes should be appropriate for the format (e.g. 10pt fonts in a paper, 24pt on a poster) and so may need to be adjusted after you resize.

It's also worth remembering that human vision isn't scale invariant: we expect a certain level of detail regardless of size.
When scaling up a tiny graphic, even losslessly, you may need to add additional detail or texture. Conversely, when shrinking graphics down, you may need to simplify.

### 3.3 Think in inches (or mm) and points, not pixels!

The size of a pixel varies depending on the screen, but the human eye stays the same!
Design your figures using real physical sizes, and leave pixels up to the screen or printer.

Define font sizes and line widths in [points](https://en.wikipedia.org/wiki/Point_(typography)), where 1 point equals 1/72nd of an inch.

If you do need to get into pixel land:
- The resolution at which a printer rasterises is given in [DPI](https://en.wikipedia.org/wiki/Dots_per_inch), or dots per inch. If you are working for screen, forget about DPI.
- The size at which raster graphics are shown on a screen depends on the pixel density, measured in [PPI](https://en.wikipedia.org/wiki/Pixel_density), or pixels per inch. 
- If you can't avoid resampling, aim to have only a single resampling step. If possible, use some integer ratio (e.g. 2 times smaller, 4 times bigger, etc.).

(Annoyingly, "PPI" is also used to mean "points per inch", i.e. 72 by the modern definition. This leads to a lot of conflation of pixels with points.)

## Examples

In the code below, we create a figure that's 4 inches wide and 2 inches high, using matplotlib. We don't tell matplotlib anything about DPI or PPI, so it uses its default settings of 100 DPI and 72 PPI. To show the image boundaries, we set a gray background colour (using the [web code](https://en.wikipedia.org/wiki/Web_colors) `#cccccc`).

In [1]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 100, 1000)
y = np.sin(x) + 1.1 * np.cos(1.2 * (x + 1.3))

f = plt.figure(figsize=(4, 2), facecolor='#cccccc')
plt.subplots_adjust(0.13, 0.22, 0.99, 0.99)
ax = f.add_subplot(1, 1, 1)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.plot(x, y)
plt.savefig('./figures-1/res-1.png')
plt.close()

We can include this image below (note that Jupyter will cache this image, so it won't change directly if you modify the code above):

![img](./figures-1/res-1.png)

We can see the pixel size of the image using e.g. PIL (the Python Image Library):

In [2]:
import PIL.Image
img = PIL.Image.open('./figures-1/res-1.png')
print(img.size)

(288, 144)


This shows us the image is 4 * 72 = 288 pixels wide, and 2 * 72 pixels high.

Now get your ruler out!
On my PC screen at home, the image measures about 3 by 1.5 inches, which is not a million miles off but not great either. On my old laptop it's 2 inches wide. If I was working on a fancier laptop or looking at my phone, it might render as just 1 inch wide.

This illustrates a first problem with raster graphics, even when staying on screen: The actual display size depends on the hardware, and more 
expensive hardware does not necessarily increase readability!

We can ask matplotlib to rasterise at a higher resolution by increasing the DPI setting used*:

(Note that we're adjusting DPI, which is for printing not for screens. This is a sign that we're doing it wrong!) 

In [3]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 100, 1000)
y = np.sin(x) + 1.1 * np.cos(1.2 * (x + 1.3))

f = plt.figure(figsize=(4, 2), facecolor='#cccccc')
plt.subplots_adjust(0.13, 0.22, 0.99, 0.99)
ax = f.add_subplot(1, 1, 1)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.plot(x, y)
plt.savefig('./figures-1/res-2.png', dpi=200)
plt.close()

This results in the following image:

![img](./figures-1/res-2.png)

Rendered 1:1 on the screen, the whole figure now looks a lot bigger, including larger fonts and thicker lines.
This is the intended behaviour: Both images are the same size (4 inches wide), and so a 1pt line in the second figure needs more pixels (or dots) than in the first.

In [4]:
img = PIL.Image.open('./figures-1/res-2.png')
print(img.size)

(800, 400)


Using PIL we see that the image size is now 4 inches * 200 DPI = 800 pixels wide, by 2 * 200 = 400 pixels high.

What if we want a figure that looks bigger on screen, but still has "normal" font sizes?
The answer is to simply tell matplotlib that we want the figure to be bigger, by increasing the figure size, without touching the DPI:

In [5]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 100, 1000)
y = np.sin(x) + 1.1 * np.cos(1.2 * (x + 1.3))

f = plt.figure(figsize=(8, 4), facecolor='#cccccc')
plt.subplots_adjust(0.07, 0.11, 0.99, 0.99)
ax = f.add_subplot(1, 1, 1)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.plot(x, y)
plt.savefig('./figures-1/res-3.png')
plt.close()

img = PIL.Image.open('./figures-1/res-3.png')
print(img.size)

(576, 288)


Now the image is wider, but fonts are rendered with the same number of pixels as in the first example:

![img](./figures-1/res-3.png)

Finally, we can avoid a lot of these steps by saving in a vector format:

In [6]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 100, 1000)
y = np.sin(x) + 1.1 * np.cos(1.2 * (x + 1.3))

f = plt.figure(figsize=(8, 4), facecolor='#cccccc')
plt.subplots_adjust(0.07, 0.11, 0.99, 0.99)
ax = f.add_subplot(1, 1, 1)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.plot(x, y)
plt.savefig('./figures-1/res-4.svg')
plt.close()

![img](./figures-1/res-4.svg)

Now get your ruler out again.
On my screen this image renders as approximately 8 1/8 inches wide, so almost exactly the intended size: the display drivers have talked to the screen (or made a good guess) of the pixel density, and so can rasterise at a level that produces the intended figure size!

Again, consider the rasterisation steps involved:

- The function is still rasterised as 1000 x and y coordinates.
- When storing, these 1000 coordinates are just written into the SVG file (so we avoid rasterisation, but do store more data than potentially shown in the end result).
- When the image is drawn on screen, the 1000 x and y coordinates are up/down sampled to whatever resolution is appropriate. The text however, is rasterised for the first and only time!
- Note how the font size in the rendered image closely matches this text!